In [219]:
import pandas as pd
from pulp import LpProblem, LpVariable, lpSum, LpMinimize, LpAffineExpression, LpConstraint
import numpy as np
import pulp as plp

In [220]:
foods = pd.read_csv('./Data_formatted/nv_df.csv', index_col='Unnamed: 0')
constraints = pd.read_csv('./Data_formatted/constraints.csv', index_col = 'Unnamed: 0')
constraints = constraints[1:] # remove calories
# Adjust amino acid requirements for bodyweight
usr_mass = input("How many kilos do you weigh?")
usr_mass = int(usr_mass)
constraints.iloc[-11:,0] *= usr_mass

How many kilos do you weigh? 76


In [221]:
# Decision Variables
food_vars  = {food:
plp.LpVariable(cat=plp.LpContinuous, 
               lowBound=0,
               name=f"servings_{food}") 
for food in foods.index.values}

# Objective Function
min_cals = plp.LpProblem("Min_Calories",plp.LpMinimize)
min_cals += lpSum([foods.calories[food]*food_vars[food] 
                    for food in foods.index.values])

In [231]:
def checkFeasibility(foods, constraints):
    # Decision Variables
    food_vars  = {food:
    plp.LpVariable(cat=plp.LpContinuous, 
                   lowBound=0,
                   name=f"servings_{food}") 
    for food in foods.index.values}

    # Objective Function
    min_cals = plp.LpProblem("Min_Calories",plp.LpMinimize)
    min_cals += lpSum([foods.calories[food]*food_vars[food] 
                        for food in foods.index.values])
    for c in constraints.index:
        if not np.isnan(constraints.loc[c][0]):
            min_cals += LpConstraint(e=lpSum([foods.loc[food,c]*food_vars[food] 
                                              for food in foods.index]),
                                 sense=plp.LpConstraintGE, 
                                 rhs=constraints.loc[c][0],
                                 name=f'min_{c}'
                                )
        if not np.isnan(constraints.loc[c][1]):
            min_cals += LpConstraint(e=lpSum([foods.loc[food,c]*food_vars[food] 
                                              for food in foods.index]),
                                 sense=plp.LpConstraintLE, 
                                 rhs=constraints.loc[c][1],
                                 name=f'max_{c}'
                                )
    min_cals.solve();
    return min_cals.status

In [226]:
constraints.shape[0]

40

In [233]:
constraints.iloc[[0,1],:]

,min,max
protein,56.0,160.0
fat_total,22.2,78.0


In [235]:
solvable = []
brk_pts = []
for i in range(0,constraints.shape[0]):
    candidates = [i for i in range(0,i+1) if i not in brk_pts]
    status = checkFeasibility(foods,constraints.iloc[candidates,:])
    if (status == -1):
        brk_pts.append(i)
    else:
        solvable.append(i)
candidates

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /Users/williammohr/opt/anaconda3/lib/python3.9/site-packages/pulp/solverdir/cbc/osx/64/cbc /var/folders/7q/63cgxnn50sl5zwxkchv_j2cr0000gn/T/90d2f1b39f1e4855a23b15e7b17cfc77-pulp.mps timeMode elapsed branch printingOptions all solution /var/folders/7q/63cgxnn50sl5zwxkchv_j2cr0000gn/T/90d2f1b39f1e4855a23b15e7b17cfc77-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 7 COLUMNS
At line 357 RHS
At line 360 BOUNDS
At line 361 ENDATA
Problem MODEL has 2 rows, 117 columns and 232 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Presolve 2 (0) rows, 109 (-8) columns and 218 (-14) elements
0  Obj 0 Primal inf 1.5918125 (1)
1  Obj 249.71325
Optimal - objective value 249.71325
After Postsolve, objective 249.71325, infeasibilities - dual 0 (0), primal 0 (0)
Optimal objective 249.7132486 - 1 iterations time 0.002, Presolve 0.00
Option f

[0,
 1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 39]

In [238]:
[c for c in range(0,40) if c not in candidates ]

[18, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38]

In [239]:
candidates

[0,
 1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 39]

In [240]:
constraints.index.values[[c for c in range(0,40) if c not in candidates ]]

array(['copper', 'chloride', 'biotin', 'chromium', 'cysteine_methionine',
       'histidine', 'leucine', 'lysine', 'methionine',
       'phenylalanine_tyrosine', 'threonine'], dtype=object)

'copper', 'chloride', 'biotin', 'chromium', 'cysteine_methionine', 'histidine', 'leucine', 'lysine', 'methionine', 'phenylalanine_tyrosine', and 'threonine' are not solvable when added to the the agglomerative model that itteratively tries to add constraints from the list.  I'll now proceed to 

In [241]:
#
constraints_micros_aa = 

,min,max
protein,56.0,160.0
fat_total,22.2,78.0
saturated_fat,0.0,12.0
cholesterol,0.0,NaN
sodium,1500.0,2300.0
choline,550.0,3500.0
folate,400.0,1000.0
niacin,16.0,35.0
pantothenic_acid,5.0,NaN
vitamin_b2,1.1,NaN


In [223]:
checkFeasibility(foods,constraints)

protein_min
protein_max
fat_total_min
fat_total_max
saturated_fat_min
saturated_fat_max
cholesterol_min
sodium_min
sodium_max
choline_min
choline_max
folate_min
folate_max
niacin_min
niacin_max
pantothenic_acid_min
vitamin_b2_min
vitamin_b1_min
vitamin_a_min
vitamin_a_max
vitamin_b12_min
vitamin_b6_min
vitamin_b6_max
vitamin_c_min
vitamin_d_mcg_min
vitamin_d_mcg_max
vitamin_k_min
calcium_min
calcium_max
copper_min
copper_max
iron_min
iron_max
magnesium_min
manganese_min
manganese_max
phosphorus_min
phosphorus_max
potassium_min
selenium_min
selenium_max
zinc_min
zinc_max
carbohydrates_min
carbohydrates_max
fiber_min
molybdenum_min
molybdenum_max
chloride_min
chloride_max
biotin_min
chromium_min
cysteine_methionine_min
histidine_min
leucine_min
lysine_min
methionine_min
phenylalanine_tyrosine_min
threonine_min
valine_min
Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /Users/williammohr/opt/anaconda3/lib/python3.9/site-packages/pulp/solverdir/cbc

'Infeasible'

In [205]:
constraints

,min,max
protein,56.0,160.0
fat_total,22.2,78.0
saturated_fat,0.0,12.0
cholesterol,0.0,NaN
sodium,1500.0,2300.0
choline,550.0,3500.0
folate,400.0,1000.0
niacin,16.0,35.0
pantothenic_acid,5.0,NaN
vitamin_b2,1.1,NaN


In [204]:
checkFeasibility(foods,constraints)

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /Users/williammohr/opt/anaconda3/lib/python3.9/site-packages/pulp/solverdir/cbc/osx/64/cbc /var/folders/7q/63cgxnn50sl5zwxkchv_j2cr0000gn/T/2f122da4a6a841928cee11adf90b977a-pulp.mps timeMode elapsed branch printingOptions all solution /var/folders/7q/63cgxnn50sl5zwxkchv_j2cr0000gn/T/2f122da4a6a841928cee11adf90b977a-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 7 COLUMNS
At line 357 RHS
At line 360 BOUNDS
At line 361 ENDATA
Problem MODEL has 2 rows, 117 columns and 232 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Presolve 2 (0) rows, 109 (-8) columns and 218 (-14) elements
0  Obj 0 Primal inf 1.5918125 (1)
1  Obj 249.71325
Optimal - objective value 249.71325
After Postsolve, objective 249.71325, infeasibilities - dual 0 (0), primal 0 (0)
Optimal objective 249.7132486 - 1 iterations time 0.002, Presolve 0.00
Option f

1

In [198]:
vitamins = ['choline', 'folate', 'niacin', 'pantothenic_acid', 'vitamin_b2',
       'vitamin_b1', 'vitamin_a', 'vitamin_b12', 'vitamin_b6', 'vitamin_c',
       'vitamin_d_mcg', 'vitamin_k', 'calcium', 'copper', 'iron', 'magnesium',
       'manganese', 'phosphorus', 'potassium', 'selenium', 'zinc']

for v in vitamins[0:5]:
    if not np.isnan(constraints.loc[v][0]):
        min_cals += LpConstraint(e=lpSum([foods.loc[food,v]*food_vars[food] 
                                          for food in foods.index]),
                             sense=plp.LpConstraintGE, 
                             rhs=constraints.loc[v][0],
                             name=f'min_{v}'
                            )
    if not np.isnan(constraints.loc[v][1]):
        min_cals += LpConstraint(e=lpSum([foods.loc[food,v]*food_vars[food] 
                                          for food in foods.index]),
                             sense=plp.LpConstraintLE, 
                             rhs=constraints.loc[v][1],
                             name=f'max_{v}'
                            )

In [200]:
min_cals.solve()
min_cals.objective.value()

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /Users/williammohr/opt/anaconda3/lib/python3.9/site-packages/pulp/solverdir/cbc/osx/64/cbc /var/folders/7q/63cgxnn50sl5zwxkchv_j2cr0000gn/T/0e6c6d9e90ee4ed390faa177aa9fe682-pulp.mps timeMode elapsed branch printingOptions all solution /var/folders/7q/63cgxnn50sl5zwxkchv_j2cr0000gn/T/0e6c6d9e90ee4ed390faa177aa9fe682-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 11 COLUMNS
At line 803 RHS
At line 810 BOUNDS
At line 811 ENDATA
Problem MODEL has 6 rows, 117 columns and 674 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Presolve 6 (0) rows, 117 (0) columns and 674 (0) elements
0  Obj 0 Primal inf 5.2153918 (3)
4  Obj 325.23604
Optimal - objective value 325.23604
Optimal objective 325.2360389 - 4 iterations time 0.002
Option for printingOptions changed from normal to all
Total time (CPU seconds):       0.00   (Wallclock se

325.236035646

In [120]:
for c in constraints.index[]:
    if not np.isnan(constraints.loc[c][0]):
        min_cals += LpConstraint(e=lpSum([foods.loc[food,c]*food_vars[food] 
                                          for food in foods.index]),
                             sense=plp.LpConstraintGE, 
                             rhs=constraints.loc[c][0],
                             name=f'min_{c}'
                            )
    if not np.isnan(constraints.loc[c][1]):
        min_cals += LpConstraint(e=lpSum([foods.loc[food,c]*food_vars[food] 
                                          for food in foods.index]),
                             sense=plp.LpConstraintLE, 
                             rhs=constraints.loc[c][1],
                             name=f'max_{c}'
                            )

In [ ]:
core_constraints = ['protein', 'cysteine_methionine', 'histidine', 'leucine', 'lysine',
       'methionine', 'phenylalanine_tyrosine', 'threonine', 'valine']

In [122]:
constraints.index

Index(['protein', 'fat_total', 'saturated_fat', 'cholesterol', 'sodium',
       'choline', 'folate', 'niacin', 'pantothenic_acid', 'vitamin_b2',
       'vitamin_b1', 'vitamin_a', 'vitamin_b12', 'vitamin_b6', 'vitamin_c',
       'vitamin_d_mcg', 'vitamin_k', 'calcium', 'copper', 'iron', 'magnesium',
       'manganese', 'phosphorus', 'potassium', 'selenium', 'zinc',
       'carbohydrates', 'fiber', 'molybdenum', 'chloride', 'biotin',
       'chromium', 'cysteine_methionine', 'histidine', 'leucine', 'lysine',
       'methionine', 'phenylalanine_tyrosine', 'threonine', 'valine'],
      dtype='object')

In [121]:
min_cals.solve()

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /Users/williammohr/opt/anaconda3/lib/python3.9/site-packages/pulp/solverdir/cbc/osx/64/cbc /var/folders/7q/63cgxnn50sl5zwxkchv_j2cr0000gn/T/9346b6b162f84f999f60d9faf9ba9323-pulp.mps timeMode elapsed branch printingOptions all solution /var/folders/7q/63cgxnn50sl5zwxkchv_j2cr0000gn/T/9346b6b162f84f999f60d9faf9ba9323-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 65 COLUMNS
At line 5949 RHS
At line 6010 BOUNDS
At line 6011 ENDATA
Problem MODEL has 60 rows, 117 columns and 5766 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Presolve determined that the problem was infeasible with tolerance of 1e-08
Analysis indicates model infeasible or unbounded
0  Obj 0 Primal inf 183679.88 (38)
11  Obj 35420182 Primal inf 18349995 (24)
Primal infeasible - objective value 35420182
PrimalInfeasible objective 35420182.37 - 11 iterations 

-1

In [ ]:
dir(plp);

In [ ]:
# Constraints

plp.Lp

In [ ]:
pulp.lpSum

In [ ]:
protein_calories = foods['protein']/foods['calories'].sort_values(ascending=False).copy()

In [ ]:
protein_calories.sort_values(ascending=False)

In [ ]:
# selecting high protein to calorie ratio foods
protein_foods = foods.loc[protein_calories.sort_values(ascending=False)[0:40].index,:]\
.query("calories > 50")

In [ ]:
constraints.drop(['saturated_fat','cholesterol'])

In [ ]:
necessary_nutrients = [c for c in constraints.index if c not in ['saturated_fat','cholesterol']]

In [ ]:
constraints.loc[~constraints.index.isin(['saturated_fat','cholesterol']),:].index.values

In [ ]:
foods[necessary_nutrients].idxmax()

In [ ]:
max_foods = foods.loc[list(set(foods[necessary_nutrients].idxmax().values)),:]

In [ ]:

# Sample data for foods and nutrients
data_foods = {
    'Food': [food for food in max_foods.index],
}

data_foods.update({col: protein_foods[col] for col in protein_foods.columns})

df_foods = pd.DataFrame(data_foods)

# Sample data for constraints
data_constraints = {
    'Nutrient': [nutrient for nutrient in constraints.index],
    'MinValue': [val for val in constraints['min']],
    'MaxValue': [val for val in constraints['max']],
}

df_constraints = pd.DataFrame(data_constraints)

# Create the LP Minimization problem
problem = LpProblem("Nutritionally_Complete_Foods", LpMinimize)

# Define decision variables
food_vars = LpVariable.dicts("Servings", df_foods['Food'], lowBound=0, cat='Integer')

# Define the objective function to minimize the total number of servings
problem += lpSum(food_vars[food] for food in df_foods['Food'])

# Add constraints based on nutrient requirements
for nutrient in df_constraints['Nutrient']:
    min_value = df_constraints.loc[df_constraints['Nutrient'] == nutrient, 'MinValue'].values[0]
    max_value = df_constraints.loc[df_constraints['Nutrient'] == nutrient, 'MaxValue'].values[0]
    
    if not pd.isna(min_value):
        problem += lpSum(food_vars[food] * df_foods.loc[df_foods['Food'] == 
                    food, nutrient].values[0] for food in df_foods['Food']) \
                    >= min_value
    if not pd.isna(max_value):
        problem += lpSum(food_vars[food] * df_foods.loc[df_foods['Food'] == 
                    food, nutrient].values[0] for food in df_foods['Food']) \
                    <= max_value

# # Add constraint for nutritional completeness
# for nutrient in df_constraints['Nutrient']:
#     min_value = df_constraints.loc[df_constraints['Nutrient'] == nutrient, 
#                             'MinValue'].values[0]
#     problem += lpSum(food_vars[food] * df_foods.loc[df_foods['Food'] == 
#                 food, nutrient].values[0] for food in df_foods['Food']) 
#                 >= min_value

# Solve the problem
problem.solve()

# Print the results
print("Status:", problem.status)

if problem.status == 1:  # If the problem was solved successfully
    for food in df_foods['Food']:
        servings = food_vars[food].value()
        if servings > 0:
            print(f"{food}: {servings} servings")
else:
    print("No solution found.")

Next I will solve to minimize calories since minimizing for serving size has no solution.

In [ ]:

# Sample data for foods and nutrients
data_foods = {
    'Food': [food for food in foods.index],
}

data_foods.update({col: foods[col] for col in foods.columns})

df_foods = pd.DataFrame(data_foods)

# Sample data for constraints
data_constraints = {
    'Nutrient': [nutrient for nutrient in constraints.index],
    'MinValue': [val for val in constraints['min']],
    'MaxValue': [val for val in constraints['max']],
}

df_constraints = pd.DataFrame(data_constraints)

# Create the LP Minimization problem
problem = LpProblem("Nutritionally_Complete_Foods", LpMinimize)

# Define decision variables
food_vars = LpVariable.dicts("Servings", df_foods['Food'], lowBound=0, cat='Continuous')

# Define the objective function to minimize the total number of servings
problem += lpSum(food_vars[food] for food in df_foods['Food'])

# Add constraints based on nutrient requirements
for nutrient in df_constraints['Nutrient']:
    min_value = df_constraints.loc[df_constraints['Nutrient'] == nutrient, 'MinValue'].values[0]
    max_value = df_constraints.loc[df_constraints['Nutrient'] == nutrient, 'MaxValue'].values[0]
    
    if not pd.isna(min_value):
        problem += lpSum(food_vars[food] * df_foods.loc[df_foods['Food'] == 
                    food, nutrient].values[0] for food in df_foods['Food']) \
                    >= min_value
    if not pd.isna(max_value):
        problem += lpSum(food_vars[food] * df_foods.loc[df_foods['Food'] == 
                    food, nutrient].values[0] for food in df_foods['Food']) \
                    <= max_value

# # Add constraint for nutritional completeness
# for nutrient in df_constraints['Nutrient']:
#     min_value = df_constraints.loc[df_constraints['Nutrient'] == nutrient, 
#                             'MinValue'].values[0]
#     problem += lpSum(food_vars[food] * df_foods.loc[df_foods['Food'] == 
#                 food, nutrient].values[0] for food in df_foods['Food']) 
#                 >= min_value

# Solve the problem
problem.solve()

# Print the results
print("Status:", problem.status)

if problem.status == 1:  # If the problem was solved successfully
    for food in df_foods['Food']:
        servings = food_vars[food].value()
        if servings > 0:
            print(f"{food}: {servings} servings")
else:
    print("No solution found.")

Looking to minimize calories:

In [ ]:

# Sample data for foods and nutrients
data_foods = {
    'Food': [food for food in foods.index],
}

data_foods.update({col: foods[col] for col in foods.columns})

df_foods = pd.DataFrame(data_foods)

# Sample data for constraints
data_constraints = {
    'Nutrient': [nutrient for nutrient in constraints.index],
    'MinValue': [val for val in constraints['min']],
    'MaxValue': [val for val in constraints['max']],
}

df_constraints = pd.DataFrame(data_constraints)

# Create the LP Minimization problem
problem = LpProblem("Nutritionally_Complete_Foods", LpMinimize)

# Define decision variables
food_vars = LpVariable.dicts("Servings", df_foods['Food'], lowBound=0, cat='Continuous')

# Define the objective function to minimize the total number of servings
ae_calories = LpAffineExpression([(food_vars[food],df_foods.loc[food,'calories']) for food in df_foods['Food']])
problem += ae_calories
# Add constraints based on nutrient requirements
for nutrient in df_constraints['Nutrient']:
    min_value = df_constraints.loc[df_constraints['Nutrient'] == nutrient, 'MinValue'].values[0]
    max_value = df_constraints.loc[df_constraints['Nutrient'] == nutrient, 'MaxValue'].values[0]
    
    if not pd.isna(min_value):
        problem += lpSum(food_vars[food] * df_foods.loc[df_foods['Food'] == 
                    food, nutrient].values[0] for food in df_foods['Food']) \
                    >= min_value
    if not pd.isna(max_value):
        problem += lpSum(food_vars[food] * df_foods.loc[df_foods['Food'] == 
                    food, nutrient].values[0] for food in df_foods['Food']) \
                    <= max_value

# # Add constraint for nutritional completeness
# for nutrient in df_constraints['Nutrient']:
#     min_value = df_constraints.loc[df_constraints['Nutrient'] == nutrient, 
#                             'MinValue'].values[0]
#     problem += lpSum(food_vars[food] * df_foods.loc[df_foods['Food'] == 
#                 food, nutrient].values[0] for food in df_foods['Food']) 
#                 >= min_value

# Solve the problem
problem.solve()

# Print the results
print("Status:", problem.status)

if problem.status == 1:  # If the problem was solved successfully
    for food in df_foods['Food']:
        servings = food_vars[food].value()
        if servings > 0:
            print(f"{food}: {servings} servings")
else:
    print("No solution found.")

In [ ]:
constraints.drop(index=['calories'],inplace=True)
# Sample data for foods and nutrients
data_foods = {
    'Food': [food for food in foods.index],
}

data_foods.update({col: foods[col] for col in foods.columns})

df_foods = pd.DataFrame(data_foods)

# Sample data for constraints
data_constraints = {
    'Nutrient': [nutrient for nutrient in constraints.index],
    'MinValue': [val for val in constraints['min']],
    'MaxValue': [val for val in constraints['max']],
}

df_constraints = pd.DataFrame(data_constraints)

# Create the LP Minimization problem
problem = LpProblem("Nutritionally_Complete_Foods", LpMinimize)

# Define decision variables
food_vars = LpVariable.dicts("Servings", df_foods['Food'], lowBound=0, cat='Continuous')

# Define the objective function to minimize the total number of servings
ae_calories = LpAffineExpression([(food_vars[food],df_foods.loc[food,'calories']) for food in df_foods['Food']])
problem += ae_calories
# Add constraints based on nutrient requirements
for nutrient in df_constraints['Nutrient']:
    min_value = df_constraints.loc[df_constraints['Nutrient'] == nutrient, 'MinValue'].values[0]
    max_value = df_constraints.loc[df_constraints['Nutrient'] == nutrient, 'MaxValue'].values[0]
    
    if not pd.isna(min_value):
        problem += lpSum(food_vars[food] * df_foods.loc[df_foods['Food'] == 
                    food, nutrient].values[0] for food in df_foods['Food']) \
                    >= min_value
    if not pd.isna(max_value):
        problem += lpSum(food_vars[food] * df_foods.loc[df_foods['Food'] == 
                    food, nutrient].values[0] for food in df_foods['Food']) \
                    <= max_value

# # Add constraint for nutritional completeness
# for nutrient in df_constraints['Nutrient']:
#     min_value = df_constraints.loc[df_constraints['Nutrient'] == nutrient, 
#                             'MinValue'].values[0]
#     problem += lpSum(food_vars[food] * df_foods.loc[df_foods['Food'] == 
#                 food, nutrient].values[0] for food in df_foods['Food']) 
#                 >= min_value

# Solve the problem
problem.solve()

# Print the results
print("Status:", problem.status)

if problem.status == 1:  # If the problem was solved successfully
    for food in df_foods['Food']:
        servings = food_vars[food].value()
        if servings > 0:
            print(f"{food}: {servings} servings")
else:
    print("No solution found.")